In [1]:
import ROOT
from ROOT import gStyle
import numpy as np

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list):
    for hist in hist_list:
        for i in range(0,len(rebining)):
            value=hist.GetBinContent(i)
            sf=hist.GetBinWidth(i)/5.0    
            hist.SetBinContent(i,value/sf)

Welcome to JupyROOT 6.20/02


In [2]:
histos={"reco_mass_bdte_btag_iso_rnn_pte_omega_mle":["y",120,5,20,"Events/5 GeV","m_{reco}(GeV)(i)",1.2],
       "reco_mass_met_outside_bdte_btag_iso_rnn_pte_omega_mle":["y",120,5,20,"Events/5 GeV","m_{reco}(GeV)(o)",1.2],
       "tau_pt_bdte_btag_iso_rnn_pte_omega_mle_mreco_inside":["y",120,5,20,"Events/5 GeV","p_{T}(#tau)(GeV)(i)",1.2],
       "tau_pt_bdte_btag_iso_rnn_pte_omega_mle_mreco_outside":["y",120,5,20,"Events/5 GeV","p_{T}(#tau)(GeV)(o)",1.2],
       "MET_bdte_btag_iso_rnn_pte_omega_mle_mreco":["y",120,5,20,"Events/5 GeV","MET(GeV)",1.2],
       "Z_pt_cuts_inside":["y",180,10,20,"Events/5 GeV","p_{T}(Z)(GeV)(i)",1.2],
       "Z_pt_cuts_outside":["y",180,10,20,"Events/5 GeV","p_{T}(Z)(GeV)(o)",1.2],
       "inva_mass_ltau_bdte_btag_iso_rnn_pte_omega_mreco":["y",120,5,20,"Events/5 GeV","m(e,#tau)(GeV)",1.2],
       "omega_bdte_btag_iso_rnn_pte_mle_mreco":["n",1,1,1,"Events/0.1","#Omega",1.2],
       "delta_phi_cuts_butphi":["n",1,1,1,"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.3],
       "delta_phi_cuts":["n",1,1,1,"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.3],
       "rnn_score_1prong_bdte_btag_iso_pte_omega_mle_mreco":["y",0.7,0.02,0.02,"Events/0.01","jetRNNScore",1.4],
       "rnn_score_3prong_bdte_btag_iso_pte_omega_mle_mreco":["y",0.7,0.02,0.02,"Events/0.01","jetRNNScore",1.4],
       "bdt_e_score_btag_iso_rnn_pte_omega_mle_mreco":["y",0.7,0.02,0.02,"Events/0.01","eBDTScore",1.2,1,0]}

In [3]:
str(histos["delta_phi_cuts"][5])

'#Delta#varphi(#tau,#mu)'

In [4]:
#CALCULATION OF RQCD
c=1
for histo in histos:
    rebin=histos[histo][0]
    hist=histo
    print(c)
    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=hist

    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)

    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()
    print("CR_OS_BEFORE")
    print(data_cr_os.Integral())
    print("CR_SS_BEFORE")
    print(data_cr_ss.Integral())
    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)
    print("CR_OS_AFTER")
    print(data_cr_os.Integral())
    print("CR_SS_AFTER")
    print(data_cr_ss.Integral())
    RQCD=data_cr_os.Integral()/data_cr_ss.Integral()

    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)

    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    if rebin=="y":
        rebining=biner(s,histos[histo][1],e,histos[histo][2],histos[histo][3])
        nb=len(rebining)-1
        mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)

    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    MJ_BG.Scale(RQCD)
    
    # SAVING A HISTOGRAM FOR MJ BG
    
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a)
    file.Close()
    
    ################################################
    
    # PLOTS
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/Signal.root","READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Zjets.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")

    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)

    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)

    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()

    hs = ROOT.THStack("hs","")
    if rebin=="y":
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)

    ratio=w.Clone()
    lista = ROOT.TList()
    lista.Add(mj)
    lista.Add(z)
    lista.Add(vv)
    lista.Add(tt)
    lista.Add(t)
    lista.Add(signal)
    ratio.Merge(lista)
    ratio.Divide(data)

    if rebin=="y":
        hist_list=[signal,data,vv,tt,t,z,w,mj]
        normalization(hist_list)

    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan)
    mj.SetLineColor(ROOT.kMagenta)
    mj.SetFillColor(ROOT.kMagenta)
    data.SetLineColor(ROOT.kBlack)

    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(mj)
    hs.Add(signal)

    gStyle.SetOptStat(1111111)

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

    pad1.Draw ()
    pad1.cd ()

    gStyle.SetStatY(0.9);                
    gStyle.SetStatX(0.9);
    gStyle.SetStatW(0.1);                
    gStyle.SetStatH(0.1); 
    if len(histos[histo])>8:
        pad1.SetLogy(histos[histo][8])

    data.Draw("pe")
    hs.Draw("HIST same")
    data.Draw("pe same")
    data.GetXaxis().Draw()

    pad1.RedrawAxis()
    data.SetTitle("Hola")
    l=ROOT.TLatex()
    l.SetNDC()
    l.DrawLatex(0.6,0.75,"Z#rightarrow#tau#tau#rightarrow e#tau")
    
    if len(histos[histo])>7:
        e=histos[histo][7]
    
    pad1.SetTicks(1,1)
    pad1.Update()
    data . GetYaxis (). SetRangeUser (1 ,histos[histo][6]*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s ,e)
    data . GetYaxis (). SetTitle (histos[histo][4])
    data . GetXaxis (). SetTitle (histos[histo][5])

    hs . GetXaxis (). SetRangeUser (s ,e)
    legend = ROOT . TLegend (0.7 ,0.68 ,0.8 ,0.88)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( mj,"MJ")

    legend . SetLineWidth (0)
    legend . Draw ()

    hs.SetTitle(a)

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.35)
    pad2.Draw ()
    pad2.cd ()
    ratio.Draw ("pe")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line . Draw (" same ")

    canvas3.Print(a+".pdf")
    c=c+1

1
CR_OS_BEFORE
2401.0
CR_SS_BEFORE
612.0
CR_OS_AFTER
769.9319484158768
CR_SS_AFTER
499.4509523234592
2
CR_OS_BEFORE
816.0
CR_SS_BEFORE
340.0
CR_OS_AFTER
233.49273587047355
CR_SS_AFTER
311.65658688676194
3
CR_OS_BEFORE
1493.0
CR_SS_BEFORE
256.0
CR_OS_AFTER
263.5228170817718
CR_SS_AFTER
204.83604855588158
4
CR_OS_BEFORE
572.0
CR_SS_BEFORE
199.0
CR_OS_AFTER
78.52674706681864
CR_SS_AFTER
182.0061240180512
5
CR_OS_BEFORE
2062.0
CR_SS_BEFORE
455.0
CR_OS_AFTER
338.25754596688785
CR_SS_AFTER
386.79048574058106
6
CR_OS_BEFORE
1478.0
CR_SS_BEFORE
255.0
CR_OS_AFTER
268.747938554734
CR_SS_AFTER
204.222105300345
7
CR_OS_BEFORE
566.0
CR_SS_BEFORE
199.0
CR_OS_AFTER
76.0722387530841
CR_SS_AFTER
182.09631102716958
8
CR_OS_BEFORE
2194.0
CR_SS_BEFORE
509.0
CR_OS_AFTER
404.58946242183447
CR_SS_AFTER
436.0739710974158
9
CR_OS_BEFORE
3581.0
CR_SS_BEFORE
1092.0
CR_OS_AFTER
891.8347905874252
CR_SS_AFTER
823.1091198474169
10
CR_OS_BEFORE
2726.0
CR_SS_BEFORE
695.0
CR_OS_AFTER
414.2043662071228
CR_SS_AFTER
593.1

Info in <TCanvas::Print>: pdf file reco_mass_bdte_btag_iso_rnn_pte_omega_mle.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file reco_mass_met_outside_bdte_btag_iso_rnn_pte_omega_mle.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file tau_pt_bdte_btag_iso_rnn_pte_omega_mle_mreco_inside.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file tau_pt_bdte_btag_iso_rnn_pte_omega_mle_mreco_outside.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file MET_bdte_btag_iso_rnn_pte_omega_mle_mreco.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file Z_pt_cuts_inside.pdf has been created
Warning in <TCanvas::Construc

In [64]:

Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
Signal=ROOT.TFile.Open("SR_OS/Signal.root","READ")
VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
Zjets=ROOT.TFile.Open("SR_OS/Zjets.root","READ")
Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")

data=Data.Get(a)
signal=Signal.Get(a)
vv=VV.Get(a)
w=Wjets.Get(a)
z=Zjets.Get(a)
tt=Ttbar.Get(a)
t=SingleTop.Get(a)
mj=MJ.Get(a)

signal.SetDirectory(0)
data.SetDirectory(0)
vv.SetDirectory(0)
w.SetDirectory(0)
z.SetDirectory(0)
tt.SetDirectory(0)
t.SetDirectory(0)
mj.SetDirectory(0)

Signal.Close()
Data.Close()
VV.Close()
Wjets.Close()
Zjets.Close()
Ttbar.Close()
SingleTop.Close()
MJ.Close()

hs = ROOT.THStack("hs","")
if rebin=="y":
    signal=signal.Rebin(nb,"signal",rebining)
    data=data.Rebin(nb,"data",rebining)
    vv=vv.Rebin(nb,"vv",rebining)
    tt=tt.Rebin(nb,"tt",rebining)
    t=t.Rebin(nb,"t",rebining)
    z=z.Rebin(nb,"z",rebining)
    w=w.Rebin(nb,"w",rebining)

ratio=w.Clone()
lista = ROOT.TList()
lista.Add(mj)
lista.Add(z)
lista.Add(vv)
lista.Add(tt)
lista.Add(t)
lista.Add(signal)
ratio.Merge(lista)
ratio.Divide(data)



if rebin=="y":
    hist_list=[signal,data,vv,tt,t,z,w,mj]
    normalization(hist_list)

w.SetFillColor(ROOT.kGreen)
w.SetLineColor(ROOT.kGreen)
z.SetFillColor(ROOT.kRed)
z.SetLineColor(ROOT.kRed)
vv.SetFillColor(ROOT.kBlue)
vv.SetLineColor(ROOT.kBlue)
tt.SetFillColor(ROOT.kYellow)
tt.SetLineColor(ROOT.kYellow)
t.SetFillColor(ROOT.kCyan)
t.SetLineColor(ROOT.kCyan)
mj.SetLineColor(ROOT.kMagenta)
mj.SetFillColor(ROOT.kMagenta)
data.SetLineColor(ROOT.kBlack)



hs.Add(vv)
hs.Add(w)
hs.Add(z)
hs.Add(tt)
hs.Add(t)
hs.Add(mj)
hs.Add(signal)




gStyle.SetOptStat(1111111)

canvas3 = ROOT.TCanvas("canvas3")
canvas3.cd()

pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

pad1.Draw ()
pad1.cd ()

gStyle.SetStatY(0.9);                
gStyle.SetStatX(0.9);
gStyle.SetStatW(0.1);                
gStyle.SetStatH(0.1); 
pad1.SetLogy(0)


data.Draw("pe")
hs.Draw("HIST same")
data.Draw("pe same")
data.GetXaxis().Draw()

pad1.RedrawAxis()
data.SetTitle("Hola")
l=ROOT.TLatex()
l.SetNDC()
l.DrawLatex(0.7,0.15,"RNNScore 3 track")

pad1.SetTicks(1,1)
pad1.Update()
data . GetYaxis (). SetRangeUser (1 ,1.5*data.GetBinContent(data.GetMaximumBin()))
data . GetXaxis (). SetRangeUser (s ,e)
data . GetYaxis (). SetTitle ("Events/0.01")
#data . GetXaxis (). SetTitle ("GeV")

hs . GetXaxis (). SetRangeUser (s ,e)
legend = ROOT . TLegend (0.7 ,0.68 ,0.8 ,0.88)
legend . AddEntry ( vv ,"Diboson")
legend . AddEntry ( data ," Data ")
legend . AddEntry ( signal ," Signal ")
legend . AddEntry ( w ," W+jets ")
legend . AddEntry ( z ," Z+jets ")
legend . AddEntry ( t ," Single top ")
legend . AddEntry ( tt ," ttbar ")
legend . AddEntry ( mj,"MJ")

legend . SetLineWidth (0)
legend . Draw ()



hs.SetTitle(a)

canvas3.cd()
pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.35)
pad2.Draw ()
pad2.cd ()
ratio.Draw ("pe")
ratio.SetStats(0)
ratio . GetYaxis (). SetRangeUser (0 ,2.0)
ratio . GetXaxis (). SetRangeUser (s ,e)
ratio . GetYaxis (). SetTitle ("MC/DATA")
ratio . GetYaxis (). SetTitleSize (0.15)
ratio . GetYaxis (). SetTitleOffset (0.25)
line = ROOT . TLine (s ,1 ,e,1)
line . SetLineColor ( ROOT . kBlack )
line . SetLineWidth (2)
line . Draw (" same ")


canvas3.Print(a+".pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file rnn_score_3prong_bdte_btag_iso_pte_omega_mle_mreco.pdf has been created
